In [1]:
import pandas as pd
from dotenv import load_dotenv
from bs4 import BeautifulSoup
import os
import json

load_dotenv()

%load_ext autoreload
%autoreload 2

### 1. Get the form we think as search

In [2]:
from helpers.forms import FormParser
from helpers.llm import YesNoMatch, QueryResultMatch, SearchInput, MatchRequest, MatchResponse
from importlib import reload


In [17]:
html_content = open('./html_files/www.smartprix.com.html', 'r').read()
soup = BeautifulSoup(html_content, 'html.parser')

forms = soup.find_all('form')

print(forms)

[<form action="/products" autocomplete="off" method="get"><div class="sm-input type-text no-label"><!-- --><input aria-label="Search" class="input empty" name="q" placeholder="Search with Smartprix" type="text" value=""/><!-- --><!-- --></div><svg class="icon i-search" style="" viewbox="0 0 24 24"><path d="M9.5,3A6.5,6.5 0 0,1 16,9.5C16,11.11 15.41,12.59 14.44,13.73L14.71,14H15.5L20.5,19L19,20.5L14,15.5V14.71L13.73,14.44C12.59,15.41 11.11,16 9.5,16A6.5,6.5 0 0,1 3,9.5A6.5,6.5 0 0,1 9.5,3M9.5,5C7,5 5,7 5,9.5C5,12 7,14 9.5,14C12,14 14,12 14,9.5C14,7 12,5 9.5,5Z"></path></svg></form>]


In [18]:
form_parser = FormParser(html_content)
forms = form_parser.parse_forms_from_html(html_content)
stripped_forms = form_parser.strip_forms_and_inputs(forms)
print(forms)
print(stripped_forms)

[{'form_index': 1, 'action': '/products', 'method': 'GET', 'id': '', 'class': [], 'inputs': [{'tag': 'input', 'name': 'q', 'type': 'text', 'value': '', 'id': '', 'required': False, 'placeholder': 'Search with Smartprix', 'css_path': 'div#app > header > div.sm-search-header > div.sm-search-bar-wrap:nth-child(1) > div.sm-search-bar > form > div.sm-input.type-text.no-label > input.input.empty'}], 'css_path': 'div#app > header > div.sm-search-header > div.sm-search-bar-wrap:nth-child(1) > div.sm-search-bar > form'}]
[{'form_index': 1, 'action': '/products', 'method': 'GET', 'id': '', 'class': [], 'inputs': [{'tag': 'input', 'name': 'q', 'type': 'text', 'value': '', 'id': '', 'required': False, 'placeholder': 'Search with Smartprix'}]}]


In [19]:
is_search_form = YesNoMatch(prompt="""
Given a form and its inputs in json parsed format, tell me if it's a search box of a website? Answer only with "True" or "False".
""")


In [20]:
import json

idx = 0
print(json.dumps(forms[idx]))
result = is_search_form.check(SearchInput(query=json.dumps(forms[idx])))
print(result.match)

{"form_index": 1, "action": "/products", "method": "GET", "id": "", "class": [], "inputs": [{"tag": "input", "name": "q", "type": "text", "value": "", "id": "", "required": false, "placeholder": "Search with Smartprix", "css_path": "div#app > header > div.sm-search-header > div.sm-search-bar-wrap:nth-child(1) > div.sm-search-bar > form > div.sm-input.type-text.no-label > input.input.empty"}], "css_path": "div#app > header > div.sm-search-header > div.sm-search-bar-wrap:nth-child(1) > div.sm-search-bar > form"}
True


### 2. Get the input we think as search

In [ ]:
websites = ['https://www.tatacliq.com', 'https://www.jiomart.com', 'https://www.boat-lifestyle.com', 'https://www.moglix.com', 'https://www.reliancedigital.in', 'https://play.google.com', 'https://www.smartprix.com', 'https://www.swiggy.com', 'https://www.flipkart.com', 'https://blinkit.com', 'https://www.bigbasket.com', 'https://www.91mobiles.com', 'https://www.poorvika.com', 'https://www.zeptonow.com', 'https://dir.indiamart.com', 'https://www.ajio.com', 'https://www.myntra.com', 'https://www.vijaysales.com', 'https://m.snapdeal.com', 'https://www.croma.com', 'https://www.nykaafashion.com']